7.1. Writing Functions That Accept Any Number of Arguments

In [1]:
def avg(first, *rest):
    return (first + sum(rest)) / (1 + len(rest))


In [2]:
avg(1, 2)

1.5

In [3]:
avg(1, 2, 3, 4, 5)

3.0

In [6]:
import html
def make_element(name, value, **attrs):
    keyvals = [' %s="%s"' % item for item in attrs.items()]
    attr_str = ''.join(keyvals)
    element = '<{name}{attrs}>{value}</{name}>'.format(name=name,
                                                      attrs=attr_str,
                                                      value=html.escape(value))
    return element

In [7]:
print(make_element('item', 'Albatross', size='large', quantity=6))

<item size="large" quantity="6">Albatross</item>


In [8]:
print(make_element('p', '<spam>'))

<p>&lt;spam&gt;</p>


In [9]:
def anyargs(*args, **kwargs):
    print(args)
    print(kwargs)
anyargs("test1", "test2", "test3", testk='testv')

('test1', 'test2', 'test3')
{'testk': 'testv'}


7.2. Writing Functions That Only Accept Keyword Arguments

In [11]:
def recv(maxsize, *, block):
    pass

In [12]:
recv(1024, True)

TypeError: recv() takes 1 positional argument but 2 were given

In [13]:
recv(1024, block=True)

In [14]:
def minimum(*values, clip=None):
    m = min(values)
    if clip is not None:
        m = clip if clip > m else m
    return m

In [15]:
minimum(1, 5, 2, -5, 10)

-5

In [16]:
minimum(1, 5, 2, -5, 10, clip=0)

0

7.3. Attaching Informational Metadata to Function Arguments

In [17]:
def add(x:int, y:int) -> int:
    return x + y

In [18]:
help(add)

Help on function add in module __main__:

add(x:int, y:int) -> int



In [20]:
add(1, 1)

2

7.4. Returning Multiple Values from a Function

7.5. Defining Functions with Default Arguments

In [ ]:
# BAD Practice
def spam(a, b=[]):
    res = []
    res.append(a)
    res.extend(b)
    return res

7.6. Defining Anonymous or Inline Functions

In [22]:
add = lambda x, y: x + y

In [23]:
add(2, 3)

5

7.7. Capturing Variables in Anonymous Functions

In [24]:
x = 10
a = lambda y: x + y
x = 20
a(15)

35

In [25]:
x = 0
a(15)

15

In [27]:
x = 10
b = lambda y, x=x: x + y
x = 20
a(15)

35

In [28]:
funcs = [lambda x: x+n for n in range(5)]
for f in funcs:
    print(f(0))

4
4
4
4
4


In [29]:
funcs = [lambda x, n=n: x+n for n in range(5)]
for f in funcs:
    print(f(0))

0
1
2
3
4


7.8. Making an N-Argument Callable Work As a Callable
with Fewer Arguments

In [31]:
def spam(a, b, c, d): 
    print(a, b, c, d)

In [33]:
from functools import partial
s1 = partial(spam, 1)
s1(2, 3, 4)

1 2 3 4


In [34]:
points = [(1, 2), (3, 4), (5, 6), (7, 8)]

In [35]:
import math
def distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return math.hypot(x2 - x1, y2 - y1)

In [36]:
pt = (4, 3)
points.sort(key=partial(distance, pt))
points

[(3, 4), (1, 2), (5, 6), (7, 8)]

In [40]:
def output_result(result, log=None): 
    if log is not None:
        log.debug('Got: %r', result)
# A sample function
def add(x, y): 
    return x + y
if __name__ == '__main__': 
    import logging
    from multiprocessing import Pool 
    from functools import partial
    logging.basicConfig(level=logging.DEBUG)
    log = logging.getLogger('test')
    p = Pool()
    p.apply_async(add, (3, 4), callback=partial(output_result, log=log))
    p.close()
    p.join()

DEBUG:test:Got: 7


7.9. Replacing Single Method Classes with Functions

7.10. Carrying Extra State with Callback Functions

In [47]:
def apply_async(func, args, *, callback):
    result = func(*args)
    callback(result)

def print_result(result):
    print('Got:', result)
    
def add(x, y):
    return x + y

In [49]:
apply_async(add, (2, 3), callback=print_result)

Got: 5


In [50]:
apply_async(add, ('hello', 'world'), callback=print_result)

Got: helloworld


In [53]:
def make_handler():
    sequence = 0
    def handler(result):
        nonlocal sequence
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))
    return handler

In [54]:
handler = make_handler()
apply_async(add, (2, 3), callback=handler)

[1] Got: 5


In [55]:
apply_async(add, ('hello', 'world'), callback=handler)

[2] Got: helloworld


In [57]:
def make_handler():
    sequence = 0
    while True:
        result = yield
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))

In [58]:
handler = make_handler()
next(handler)
apply_async(add, (2, 3), callback=handler.send)

[1] Got: 5


In [59]:
apply_async(add, ('hello', 'world'), callback=handler.send)

[2] Got: helloworld


7.11. Inlining Callback Functions

In [63]:
from queue import Queue
from functools import wraps

class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args

def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func, a.args, callback=result_queue.put)
            except StopIteration:
                break
    return wrapper
    
@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)
    r = yield Async(add, ('hello', 'world'))
    print(r)
    for n in range(10):
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')

In [64]:
test()

5
helloworld
0
2
4
6
8
10
12
14
16
18
Goodbye


7.12. Accessing Variables Defined Inside a Closure

In [68]:
def sample():
    n = 0
    def func():
        print('n=', n)
        
    def get_n():
        return n
    
    def set_n(value):
        nonlocal n
        n = value
        
    func.get_n = get_n
    func.set_n = set_n
    return func

In [69]:
f = sample()
f()

n= 0


In [70]:
f.set_n(10)
f()

n= 10
